_**DELETE BEFORE PUBLISHING**_

_This is a template also containing the style guide for use cases. The styling uses the use-case css when uploaded to the website, which will not be visible on your local machine._

_Change any text marked with {} and delete any cells marked DELETE_

***

In [5]:
# DELETE BEFORE PUBLISHING
# This is just here so you can preview the styling on your local machine

from IPython.core.display import HTML
HTML("""
<style>
.usecase-title, .usecase-duration, .usecase-section-header {
    padding-left: 15px;
    padding-bottom: 10px;
    padding-top: 10px;
    padding-right: 15px;
    background-color: #0f9295;
    color: #fff;
}

.usecase-title {
    font-size: 1.7em;
    font-weight: bold;
}

.usecase-authors, .usecase-level, .usecase-skill {
    padding-left: 15px;
    padding-bottom: 7px;
    padding-top: 7px;
    background-color: #baeaeb;
    font-size: 1.4em;
    color: #121212;
}

.usecase-level-skill  {
    display: flex;
}

.usecase-level, .usecase-skill {
    width: 50%;
}

.usecase-duration, .usecase-skill {
    text-align: right;
    padding-right: 15px;
    padding-bottom: 8px;
    font-size: 1.4em;
}

.usecase-section-header {
    font-weight: bold;
    font-size: 1.5em;
}

.usecase-subsection-header, .usecase-subsection-blurb {
    font-weight: bold;
    font-size: 1.2em;
    color: #121212;
}

.usecase-subsection-blurb {
    font-size: 1em;
    font-style: italic;
}
</style>
""")

<div class="usecase-title">Endemic Species Planting  in laneways</div>

<div class="usecase-authors"><b>Authored by: </b>Alison Collins</div>

<div class="usecase-duration"><b>Duration:</b>90 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>beginner</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>
</div>

<div class="usecase-section-header">Scenario</div>

I am a resident living in Melbourne who wishes to take part in the greening Melbourne's laneways program. I want to know the best native species to plant near me.

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- know the endemic plants suited to the laneways

<div class="usecase-section-header">Why the interest in laneway greenery?<div>

The benefits of planting vegetation in cities includes:
- oxygen production for fresh air to breathe
- storage of carbon dioxide, helping reduce green house gases
- trees absorb water, helping reduce flooding events
- evapotranspiration from leaves produces a cooling effect, reducing heat waves
- provide habitats for wildlife
- trees reduce cortisol in our brains, improving mental health and wellbeing
More information:
https://www.treesforcities.org/stories/trees-in-our-cities-10-reasons-we-need-to-plant-more

Why plant native species?
- these species are naturally homes to native wildlife, so encourging native wildlife numbers
- they are adapted to the environment, requiring less time in maintinence
- helps prevent biodiversity, reduing species loss and regaining historical species balance
More information: https://resources.austplants.com.au/information/gardening-with-native-plants/






<div class="usecase-section-header">Relevant datasets<div>

[Pre-Colonial Plant List](https://data.melbourne.vic.gov.au/explore/dataset/pre-colonial-plant-list/export/)

- The flora of Melbourne species list was compared again histroical observation records and specimens to determine historical occurence of flora in the city. Plant names as well as goring environment can be found in this dataset.

[Laneways with Greening Potential](https://data.melbourne.vic.gov.au/explore/dataset/laneways-with-greening-potential/export/)
- The City of Melbourne collected data analysing the best potential type of plantings in each of the city laneways, farm platings for edible plants, park species, vertically goring species and forest type species.





In [50]:
#IMPORT DATASETS
# Base url for v2.1 api
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'

# PLANT LIST DATASET
# Name of dataset to import
PLANT_DATASET = 'pre-colonial-plant-list'
# Set number of records to pull in at a time
NUMBER_OF_RECORDS = 100
# Set offset
OFFSET = 0
# Include any further filters needed
FILTERS = f'records?limit={NUMBER_OF_RECORDS}&offset={OFFSET}&timezone=UTC'
# Set plant URL
PLANT_URL = f'{BASE_URL}{PLANT_DATASET}/{FILTERS}'
# Print plant url to check it is correct
PLANT_URL

'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/pre-colonial-plant-list/records?limit=100&offset=0&timezone=UTC'

In [51]:
# Requst plant data using get request
plant_result = requests.get(PLANT_URL)
#check for response code to see if data was collected
#200 means data was collected
plant_result

<Response [200]>

In [52]:
plant_result.text

'{"total_count": 1067, "results": [{"species": "Acacia mucronata var. longifolia", "common_name_s": "Narrow-leaf Wattle, Variable Sallow Wattle", "certain": null, "beach_and_dunes": 0.0, "saltmarsh": 0.0, "coastal_marshlands_and_brackish_flats": 0.0, "swamp_scrub": 0.0, "woodlands_and_heathlands_on_sand": 0.0, "woodlands_and_forests_on_sedimentary_hills_valleys_and_ridges": 1.0, "grasslands_and_woodlands_on_fertile_plains": 0.0, "cliffs_and_escarpments": 0.0, "river_banks_and_creeklines": 0.0, "wet_heathland": 0.0, "freshwater_wetland": 0.0, "saltwater_wetland": 0, "most_likely_occurrence": 1.0, "epbc_category_of_threat": null, "ffg_extinction_risk": null, "ffg_category_of_threat": null}, {"species": "Acacia pycnantha", "common_name_s": "Golden Wattle", "certain": 1, "beach_and_dunes": 0.0, "saltmarsh": 0.0, "coastal_marshlands_and_brackish_flats": 0.0, "swamp_scrub": 1.0, "woodlands_and_heathlands_on_sand": 2.1, "woodlands_and_forests_on_sedimentary_hills_valleys_and_ridges": 3.2, "gr

In [56]:
# Convert results section of data into a JSON file (keeping results key only)
result_json = plant_result.json()
plants = result_json["results"]
plants

[{'species': 'Acacia mucronata var. longifolia',
  'common_name_s': 'Narrow-leaf Wattle, Variable Sallow Wattle',
  'certain': None,
  'beach_and_dunes': 0.0,
  'saltmarsh': 0.0,
  'coastal_marshlands_and_brackish_flats': 0.0,
  'swamp_scrub': 0.0,
  'woodlands_and_heathlands_on_sand': 0.0,
  'woodlands_and_forests_on_sedimentary_hills_valleys_and_ridges': 1.0,
  'grasslands_and_woodlands_on_fertile_plains': 0.0,
  'cliffs_and_escarpments': 0.0,
  'river_banks_and_creeklines': 0.0,
  'wet_heathland': 0.0,
  'freshwater_wetland': 0.0,
  'saltwater_wetland': 0,
  'most_likely_occurrence': 1.0,
  'epbc_category_of_threat': None,
  'ffg_extinction_risk': None,
  'ffg_category_of_threat': None},
 {'species': 'Acacia pycnantha',
  'common_name_s': 'Golden Wattle',
  'certain': 1,
  'beach_and_dunes': 0.0,
  'saltmarsh': 0.0,
  'coastal_marshlands_and_brackish_flats': 0.0,
  'swamp_scrub': 1.0,
  'woodlands_and_heathlands_on_sand': 2.1,
  'woodlands_and_forests_on_sedimentary_hills_valleys_an

In [58]:
# Make a pandas dataframe from json data
plant_df = pd.json_normalize(plants)
plant_df

,species,common_name_s,certain,beach_and_dunes,saltmarsh,coastal_marshlands_and_brackish_flats,swamp_scrub,woodlands_and_heathlands_on_sand,woodlands_and_forests_on_sedimentary_hills_valleys_and_ridges,grasslands_and_woodlands_on_fertile_plains,cliffs_and_escarpments,river_banks_and_creeklines,wet_heathland,freshwater_wetland,saltwater_wetland,most_likely_occurrence,epbc_category_of_threat,ffg_extinction_risk,ffg_category_of_threat
0,Acacia mucronata var. longifolia,"Narrow-leaf Wattle, Variable Sallow Wattle",NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1.0,None,None,None
1,Acacia pycnantha,Golden Wattle,1.0,0.0,0.0,0.0,1.0,2.1,3.2,3.1,2.1,2.1,0.0,0.0,0,3.2,None,None,None
2,Acacia verticillata var. verticillata,Prickly Moses,NaN,0.0,0.0,0.0,3.1,3.1,0.0,0.0,0.0,3.1,1.0,0.0,0,3.1,None,None,None
3,Acaena echinata,Sheep's Burr,1.0,0.0,0.0,0.0,0.0,1.0,3.1,3.1,3.1,2.1,0.0,0.0,0,3.1,None,None,None
4,Acrotriche prostrata,Trailing Ground-Berry,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Corybas fimbriatus,Fringed Helmet-orchid,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0,1.0,None,Victoria,Endangered
96,Cotula australis,Common Cotula,NaN,1.0,0.0,2.1,2.1,3.1,3.1,3.1,2.1,2.1,1.0,0.0,0,3.1,None,None,None
97,Craspedia paludicola,Swamp Billy-buttons,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,None,None,None
98,Craspedia sp. 2,Derrinallum Billy-buttons,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1.0,None,Victoria,Endangered


In [13]:
# Function that extracts the data from the JSON 
def get_data(base, data_url, offset = 0):
    
    filters = f'records?limit={100}&offset={offset}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result = requests.get(url)
    # if you need to extract more than 9900 records, enter the api key like below
    #result = requests.get(url + f'&apikey={API_KEY}')
    result_json = result.json()
    max_results = result_json['total_count']
    #links = result_json['links']
    records = result_json['results']
    records_df = pd.json_normalize(records)
    
    # fix columns names
    #records_df.drop(columns=['links'],inplace=True)
    column_names = records_df.columns.values.tolist()
    
    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names
    
    next_url = None
    
    # get next url
    #if records_df.shape[0] != max_results:
        #for l in links:
           # if l['rel']=='next':
               # next_url = l['href']
                
    return [records_df, next_url, column_names]


# Appends each set of data 
def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df


def extract_dataframe(dataset):
    df_new, url_next, column_names = get_data(base_url, dataset)
    df = pd.DataFrame(columns=column_names)
    while url_next != None:
        df_new, url_next, column_names = get_data(base_url, dataset, df.shape[0])
        df = add_to_dataframe(df, df_new)
        # This limits the dataset to only 9900 entries, should be removed if you want to import more data
        # To get more than 9900, enter the API key in the get data function
        if df.shape[0] == 9900:
            break
    return df




df = extract_dataframe("pre-colonial-plant-list")
df

,species,common_name_s,certain,beach_and_dunes,saltmarsh,coastal_marshlands_and_brackish_flats,swamp_scrub,woodlands_and_heathlands_on_sand,woodlands_and_forests_on_sedimentary_hills_valleys_and_ridges,grasslands_and_woodlands_on_fertile_plains,cliffs_and_escarpments,river_banks_and_creeklines,wet_heathland,freshwater_wetland,saltwater_wetland,most_likely_occurrence,epbc_category_of_threat,ffg_extinction_risk,ffg_category_of_threat


***

_**DELETE BEFORE PUBLISHING**_

## Style guide for use cases

### Headers

For styling within your markdown cells, there are two choices you can use for headers.

1) You can use HTML classes specific to the use case styling:

```<p class="usecase-subsection-header">This is a subsection header.</p>```

<p style="font-weight: bold; font-size: 1.2em;">This is a subsection header.</p>

```<p class="usecase-subsection-blurb">This is a blurb header.</p>```

<p style="font-weight: bold; font-size: 1em; font-style:italic;">This is a blurb header.</p>


2) Or if you like you can use the markdown header styles:

```# for h1```

```## for h2```

```### for h3```

```#### for h4```

```##### for h5```

## Plot colour schemes

General advice:
1. Use the same colour or colour palette throughout your notebook, unless variety is necessary
2. Select a palette based on the type of data being represented
3. Consider accessibility (colourblindness, low vision)

#### 1) If all of your plots only use 1-2 colors use one of the company style colors:

| Light theme | Dark Theme |
|-----|-----|
|<p style="color:#2af598;">#2af598</p>|<p style="color:#08af64;">#08af64</p>|
|<p style="color:#22e4ac;">#22e4ac</p>|<p style="color:#14a38e;">#14a38e</p>|
|<p style="color:#1bd7bb;">#1bd7bb</p>|<p style="color:#0f9295;">#0f9295</p>|
|<p style="color:#14c9cb;">#14c9cb</p>|<p style="color:#056b8a;">#056b8a</p>|
|<p style="color:#0fbed8;">#0fbed8</p>|<p style="color:#121212;">#121212</p>|
|<p style="color:#08b3e5;">#08b3e5</p>||


#### 2) If your plot needs multiple colors, choose an appropriate palette using either of the following tutorials:
- https://seaborn.pydata.org/tutorial/color_palettes.html
- https://matplotlib.org/stable/tutorials/colors/colormaps.html

#### 3) Consider accessibility as well.

For qualitative plotting Seaborn's 'colorblind' palette is recommended. For maps with sequential or diverging it is recommended to use one of the Color Brewer schemes which can be previewed at https://colorbrewer2.org/.

If you want to design your own colour scheme, it should use the same principles as Cynthia Brewer's research (with variation not only in hue but also, saturation or luminance).

### References

Be sure to acknowledge your sources and any attributions using links or a reference list.

If you have quite a few references, you might wish to have a dedicated section for references at the end of your document, linked using footnote style numbers.

You can connect your in-text reference by adding the number with a HTML link: ```<a href="#fn-1">[1]</a>```

and add a matching ID in the reference list using the ```<fn>``` tag: ```<fn id="fn-1">[1] Author (Year) _Title_, Publisher, Publication location.</fn>```